In [1]:
# !pip3 install ray ray[client] --user
# !pip3 install datasets --user

In [2]:
from transformers import (
    AutoConfig, 
    AutoModelForCausalLM, 
    AutoTokenizer, 
    default_data_collator,
    TrainingArguments,
    Trainer,
    TrainerCallback
)
from datasets import load_dataset, load_from_disk

import ray
from ray import train
from ray.train import Checkpoint
from ray.train.huggingface import TransformersTrainer
from ray.train import ScalingConfig
from ray.train.torch import TorchTrainer
from transformers.trainer_utils import get_last_checkpoint
from ray.train.huggingface.transformers import RayTrainReportCallback
from streaming.base.format.mds.encodings import Encoding, _encodings
from streaming import StreamingDataset
import streaming
import torch
import numpy as np
import s3fs
import os
import shutil
import wandb

wandb_api = wandb.Api()

/home/jovyan/.local/lib/python3.10/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: 'libc10_cuda.so: cannot open shared object file: No such file or directory'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


In [3]:
WANDB_PROJECT = os.environ.get('WANDB_PROJECT', 'run-ray')
WANDB_API_KEY = os.environ.get('WANDB_API_KEY', wandb_api.api_key)
MINIO = os.environ.get('MINIO', 'http://minio:9000')

In [4]:
fs = s3fs.S3FileSystem(endpoint_url = MINIO, anon = True)

In [5]:
s3_output_dir = 'train/gpt2-test'
checkpoints = fs.ls(s3_output_dir)
checkpoints = [f for f in checkpoints if 'checkpoint-' in f]
checkpoints = sorted(checkpoints, key = lambda x: int(x.split('-')[-1]))
checkpoints

['train/gpt2-test/checkpoint-100', 'train/gpt2-test/checkpoint-200']

In [6]:
output_temp = '/home/jovyan/temp'
checkpoint = checkpoints[-1]
shutil.rmtree(output_temp, ignore_errors = True)
fs.get(checkpoint, output_temp, recursive = True)

[None, None, None, None, None, None, None, None, None, None]

In [7]:
def train_func(config):
    
    MINIO = os.environ.get('MINIO', 'http://minio:9000')
    fs = s3fs.S3FileSystem(endpoint_url = MINIO, anon = True)

    class UInt16(Encoding):
        def encode(self, obj) -> bytes:
            return obj.tobytes()

        def decode(self, data: bytes):
            return np.frombuffer(data, np.uint16)

    _encodings['uint16'] = UInt16
    
    class DatasetFixed(torch.utils.data.Dataset):
        def __init__(self, local, remote):
            
            streaming.base.util.clean_stale_shared_memory()
            self.dataset = StreamingDataset(local=local, remote = remote)

        def __getitem__(self, idx):
            data = self.dataset[idx]
            data['labels'] = data["input_ids"].copy()
            data.pop('token_type_ids', None)
            for k in data.keys():
                data[k] = data[k].astype(np.int64)
            return data

        def __len__(self):
            return len(self.dataset)

    train_dataset = DatasetFixed(local=config['local'], remote=config['remote'])
    
    MODEL_NAME = "gpt2"
    model = AutoModelForCausalLM.from_pretrained(MODEL_NAME)
    directory = f'{MODEL_NAME}-test'
    
    output_dir = os.path.join('/home/ubuntu/storage', directory)
    output_temp = os.path.join('/home/ubuntu/storage', 'temp')
    s3_bucket = 'train'
    s3_output_dir = os.path.join(s3_bucket, directory)
    
    args = TrainingArguments(
        output_dir=output_dir,
        save_strategy="steps",
        logging_strategy="steps",
        learning_rate=2e-5,
        weight_decay=0.01,
        max_steps=10000,
        save_steps = 100,
        save_total_limit = 2,
        logging_steps = 1,
        per_device_train_batch_size = 6,
    )

    trainer = Trainer(
        model=model,
        args=args,
        train_dataset=train_dataset,
        data_collator=default_data_collator,
    )
    
    
    class S3Callback(TrainerCallback):
        def on_save(self, args, state, control, **kwargs):
            fs.delete(s3_output_dir)
            fs.put(output_dir, s3_bucket, recursive=True)
        
    trainer.add_callback(S3Callback())
    
    checkpoints = []
    try:
        checkpoints = fs.ls(s3_output_dir)
        checkpoints = [f for f in checkpoints if 'checkpoint-' in f]
        checkpoints = sorted(checkpoints, key = lambda x: int(x.split('-')[-1]))
    except:
        pass
    
    if len(checkpoints):
        checkpoint = checkpoints[-1]
        print(f'load checkpoint from {checkpoint}')
        shutil.rmtree(output_temp, ignore_errors = True)
        fs.get(checkpoint, output_temp, recursive = True)
        trainer.train(resume_from_checkpoint=output_temp)
    else:
        trainer.train()

In [8]:
runtime_env = {
    'pip': ['wandb', 's3fs', 'mosaicml-streaming'],
    'env_vars': {
        'WANDB_PROJECT': WANDB_PROJECT,
        'WANDB_API_KEY': WANDB_API_KEY,
        'S3_ENDPOINT_URL': MINIO,
        'MINIO': MINIO,
    }
}

In [9]:
class RayConnection:
    def __init__(self, address, **kwargs):
        ray.init(address=address, **kwargs)

    def __enter__(self):
        return self

    def __exit__(self, typ, value, traceback):
        ray.shutdown()

In [ ]:
import time

while True:
    with RayConnection("ray://ray-master:10001", runtime_env=runtime_env):
        scaling_config = ScalingConfig(num_workers=2, use_gpu=True)
        run_config = train.RunConfig(failure_config=train.FailureConfig(max_failures=-1))
        ray_trainer = TorchTrainer(
            train_func,
            train_loop_config={
                'local': 'local_dir',
                'remote': 's3://train/indexed'
            },
            scaling_config=scaling_config,
            run_config=run_config

        )
        result = ray_trainer.fit()
    
    print('ray cluster disconnected, time to sleep.')
    time.sleep(10)

(TunerInternal pid=26850) [output] This will use the new output engine with verbosity 1. To disable the new output and use the legacy output engine, set the environment variable RAY_AIR_NEW_OUTPUT=0. For more information, please see https://github.com/ray-project/ray/issues/36949


(TunerInternal pid=26850) 
(TunerInternal pid=26850) View detailed results here: /home/ubuntu/ray_results/TorchTrainer_2023-11-22_07-45-45
(TunerInternal pid=26850) To visualize your results with TensorBoard, run: `tensorboard --logdir /home/ubuntu/ray_results/TorchTrainer_2023-11-22_07-45-45`


(TunerInternal pid=26850) AIR_VERBOSITY is set, ignoring passed-in ProgressReporter for now.


(TunerInternal pid=26850) 
(TunerInternal pid=26850) Training started with configuration:
(TunerInternal pid=26850) ╭───────────────────────────────────────────────╮
(TunerInternal pid=26850) │ Training config                               │
(TunerInternal pid=26850) ├───────────────────────────────────────────────┤
(TunerInternal pid=26850) │ train_loop_config/local             local_dir │
(TunerInternal pid=26850) │ train_loop_config/remote   s3://train/indexed │
(TunerInternal pid=26850) ╰───────────────────────────────────────────────╯


(TorchTrainer pid=3610, ip=10.208.0.196) Starting distributed worker processes: ['3653 (10.208.0.196)', '27023 (10.208.0.206)']
(RayTrainWorker pid=3653, ip=10.208.0.196) Setting up process group for: env:// [rank=0, world_size=2]


...

(RayTrainWorker pid=3755, ip=10.208.0.196) {'loss': 8.8191, 'learning_rate': 1.9356000000000002e-05, 'epoch': 0.03}
(RayTrainWorker pid=27656) {'loss': 8.8191, 'learning_rate': 1.9356000000000002e-05, 'epoch': 0.03}


  3%|▎         | 323/10000 [02:02<1:20:05,  2.01it/s]


(RayTrainWorker pid=3755, ip=10.208.0.196) {'loss': 7.3698, 'learning_rate': 1.9354e-05, 'epoch': 0.03}
(RayTrainWorker pid=27656) {'loss': 7.3698, 'learning_rate': 1.9354e-05, 'epoch': 0.03}
(RayTrainWorker pid=27656) {'loss': 8.2461, 'learning_rate': 1.9352e-05, 'epoch': 0.03}
(RayTrainWorker pid=3755, ip=10.208.0.196) {'loss': 8.2461, 'learning_rate': 1.9352e-05, 'epoch': 0.03}


  3%|▎         | 325/10000 [02:03<1:19:44,  2.02it/s]


(RayTrainWorker pid=3755, ip=10.208.0.196) {'loss': 7.3536, 'learning_rate': 1.9350000000000003e-05, 'epoch': 0.03}
(RayTrainWorker pid=27656) {'loss': 7.3536, 'learning_rate': 1.9350000000000003e-05, 'epoch': 0.03}
